In [13]:
%load_ext autoreload
%autoreload 2



import os
import pickle

import numpy as np

from utils import get_dataset_from_file
from run_experiment.gas_experiment import run_gas_experiment
from default_parameters import *

from gluonts.mx.distribution import StudentTOutput, MultivariateGaussianOutput
from gluonts.evaluation import make_evaluation_predictions, Evaluator, MultivariateEvaluator
from gluonts.mx.trainer import Trainer
from gluonts.mx.trainer.callback import TrainingHistory
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
import json
from pathlib import Path
from gluonts.dataset.split import split
from gluonts.dataset.common import ListDataset

import mxnet as mx
from my_models.gluonts_models.univariate.feedforward_linear_means._estimator import (
    SimpleFeedForwardEstimator as FF_gluonts_univariate_linear,
)
from my_models.gluonts_models.univariate.feedforward_gas_means._estimator import (
    SimpleFeedForwardEstimator as FF_gluonts_univariate_gas,
)
from my_models.gluonts_models.univariate.feedforward_gas_means_point._estimator import (
    SimpleFeedForwardEstimator as FF_gluonts_univariate_gas_point,
)
from my_models.gluonts_models.feedforward_multivariate_linear_means._estimator import (
    SimpleFeedForwardEstimator as FF_gluonts_multivariate_linear,
)
# from my_models.gluonts_models.multivariate_feedforward_gas_means._estimator import (
#     SimpleFeedForwardEstimator as FF_gluonts_multivariate_gas,
# )

from my_models.gluonts_models.univariate.deepar_gas_means._estimator import (
    DeepAREstimator as DeepAR_gluonts_univariate_gas,
)

from my_models.gluonts_models.univariate.transformer_gas_means._estimator import (
    TransformerEstimator as Transformer_gluonts_gas_means,
)

from run_experiment.dl_model_experiment import GasHybridBlock
from normalizer import GASNormalizer
from sklearn.linear_model import LinearRegression

import time
import optuna
import argparse

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:

seasonality = {
    'nn5_weekly': 52.17857142857143,
    'us_births_dataset': 7,
    'weather': 7,
    'sunspot_without_missing': 7,
    'solar_10_minutes': 144,
    'hospital': 12,
    'rideshare_without_missing': 24,
    'fred_md': 12
}
class Objective:

    def __init__( self, MODEL, DATASET_NAME, ctx, DATASET_FILE_FOLDER, multivariate=False, mean_str=0.1, var_str=0.1):
        # self.train, self.test, self.freq, self.seasonality = get_dataset_from_file(f'{ROOT_FOLDER}/tsf_data/{DATASET_NAME}',prediction_length,context_length)
        self.model = MODEL
        self.multivariate = multivariate
        self.ctx = ctx
        self.dataset_name = DATASET_NAME

        self.seasonality = seasonality[DATASET_NAME]

        DATASET_TYPE = "gluonts"  # "synthetic"
        DATASET_PARAMS = real_world_data_params  # synthetic_generation_params
        DATASET_PARAMS["multivariate"] = self.multivariate
        # DATASET_FILE_FOLDER = None #'tsf_data'

        NORMALIZER_NAME = "gas_t_student"  # "gas_simple_gaussian", "gas_complex_gaussian"
        NORMALIZER_INITIAL_GUESSES = gas_t_stud_initial_guesses  # gas_{name}_*
        NORMALIZER_BOUNDS = gas_t_stud_bounds
        NORMALIZER_PARAMS = gas_t_stud_params
        NORMALIZER_PARAMS['mean_strength'] = mean_str
        NORMALIZER_PARAMS['var_strength'] = var_str

        self.mean_str = mean_str
        self.var_str = var_str


        MEAN_LAYER_NAME = "gas"  # TODO: gas
        MEAN_LAYER_PARAMS = gas_mean_layer_params

        DL_MODEL_LIBRARY = "gluonts"  # "torch"
        DL_MODEL_NAME = MODEL  # TODO: "transformer"
        if self.model == 'feedforward':
            DL_MODEL_PARAMS = gluonts_feedforward_params
        elif self.model == 'transformer':
            DL_MODEL_PARAMS = gluonts_transformer_params
        elif self.model == 'deepar':
            DL_MODEL_PARAMS = gluonts_deepar_params

        N_TRAINING_SAMPLES = 5000
        N_TEST_SAMPLES = 1000

        ROOT_FOLDER = (
            f"RESULTS_{DATASET_NAME}_{self.model}_{NORMALIZER_NAME}_{MEAN_LAYER_NAME}_{DL_MODEL_LIBRARY}_{mean_str}_{var_str}_POINTTEST"
        )
        if DATASET_PARAMS["multivariate"]:
            ROOT_FOLDER += "_multivariate"

        # run for the first time to get all the means and stuff
        training_params = run_gas_experiment(
          DATASET_NAME,
          DATASET_TYPE,
          DATASET_PARAMS,
          ROOT_FOLDER,
          NORMALIZER_NAME,
          NORMALIZER_INITIAL_GUESSES,
          NORMALIZER_BOUNDS,
          MEAN_LAYER_NAME,
          DL_MODEL_LIBRARY,
          DL_MODEL_NAME,
          DATASET_FILE_FOLDER,
          NORMALIZER_PARAMS,
          MEAN_LAYER_PARAMS,
          DL_MODEL_PARAMS,
          N_TRAINING_SAMPLES,
          N_TEST_SAMPLES,
        )

        # get the parameters needed to run the model part

        self.n_features, self.context_length, self.prediction_length, self.freq, self.dataset, self.mean_layer, self.dl_model_name, self.dl_model_params, self.folders = training_params

        self.train_original, self.test = self.dataset


        self.train, test_template = split(self.train_original, offset=-self.prediction_length)
        validation = test_template.generate_instances(
            prediction_length=self.prediction_length,
        )
        # Assuming `validation` is a list of (input, output) pairs
        validation_data = [
            {
                "start": v[0]["start"],  # replace with the actual start time
                "target": np.concatenate([v[0]['target'], v[1]['target']]),
                "feat_dynamic_real": v[0]["feat_dynamic_real"],
                "feat_static_real": v[0]["feat_static_real"],
            }
            for v in validation
        ]

        self.validation = ListDataset(validation_data, freq=self.freq)

    def get_params(self, trial) -> dict:

        if self.model == 'feedforward':
          return {
              "num_hidden_dimensions": [trial.suggest_int("hidden_dim_{}".format(i), 10, 100) for i in range(trial.suggest_int("num_layers", 1, 5))],
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-5, 1e-1),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
          }
        elif self.model == 'deepar':
           return {
              "num_cells": trial.suggest_int("num_cells", 10, 100),
              "num_layers": trial.suggest_int("num_layers", 1, 10), # was 1, 5
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3), # was 1e-6, 1e-4
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100)
           }
        elif self.model == 'transformer':
          # num_heads must divide model_dim
          valid_pairs = [ (i,d) for i in range(10,101) for d in range(1,11) if i%d == 0  ]
          model_dim_num_heads_pair = trial.suggest_categorical("model_dim_num_heads_pair", valid_pairs)

          return {
              "inner_ff_dim_scale": trial.suggest_int("inner_ff_dim_scale", 1, 5),
              "model_dim": model_dim_num_heads_pair[0],
              "embedding_dimension": trial.suggest_int("embedding_dimension", 1, 10),
              "num_heads": model_dim_num_heads_pair[1],
              "dropout_rate": trial.suggest_uniform("dropout_rate", 0.0, 0.5),
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-5, 1e-1),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
          }

    def __call__(self, trial):

        params = self.get_params(trial)

        return self.train_and_test(params)

    def train_and_test(self, params, save=False):

        history = TrainingHistory()
        trained_mean_layer = self.mean_layer
        if isinstance(trained_mean_layer, LinearRegression):
            mean_layer = mx.gluon.nn.HybridSequential()
            mean_layer.add(
                mx.gluon.nn.Dense(
                    units=self.prediction_length * self.n_features,
                    weight_initializer=mx.init.Constant(trained_mean_layer.coef_),
                    bias_initializer=mx.init.Constant(trained_mean_layer.intercept_),  # type: ignore # bias is a numpy array, don't know the reasons for this typing error
                )
            )
            mean_layer.add(
                mx.gluon.nn.HybridLambda(
                    lambda F, o: F.reshape(
                        o, (-1, self.prediction_length * self.n_features)
                    )  # no need for that but just to be sure
                )
            )
        elif isinstance(trained_mean_layer, GASNormalizer):
            mean_layer = GasHybridBlock(trained_mean_layer, self.n_features, self.prediction_length)
        else:
            raise ValueError(
                f"Unknown mean layer type: {type(trained_mean_layer)} {trained_mean_layer}"
            )

        # freeze the parameters
        for param in mean_layer.collect_params().values():
            param.grad_req = "null"

        if self.model == 'feedforward' and self.multivariate:
            if isinstance(trained_mean_layer, GASNormalizer):
                estimator = FF_gluonts_multivariate_gas(
                    mean_layer,
                    self.n_features,
                    MultivariateGaussianOutput(dim=self.n_features),
                    prediction_length=self.prediction_length,
                    context_length=self.context_length,
                    num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
                    trainer=Trainer(hybridize=False,ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                     num_batches_per_epoch=100, callbacks=[history]),
                    
                )
            else:
                estimator = FF_gluonts_multivariate_linear(
                    mean_layer,
                    self.n_features,
                    MultivariateGaussianOutput(dim=self.n_features),
                    prediction_length=self.prediction_length,
                    context_length=self.context_length,
                    num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
                    trainer=Trainer(hybridize=False,ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                     num_batches_per_epoch=100, callbacks=[history]),
                )
        elif self.model == 'feedforward' and not self.multivariate:
            if isinstance(trained_mean_layer, GASNormalizer):
                # estimator = FF_gluonts_univariate_gas(
                #     mean_layer,
                #     distr_output=StudentTOutput(),
                #     prediction_length=self.prediction_length,
                #     context_length=self.context_length,
                #     num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
                #     trainer=Trainer(hybridize=False,ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                #                      num_batches_per_epoch=100, callbacks=[history]),
                # )
                estimator = FF_gluonts_univariate_gas_point(
                    mean_layer,
                    distr_output=StudentTOutput(),
                    prediction_length=self.prediction_length,
                    context_length=self.context_length,
                    num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
                    trainer=Trainer(hybridize=False,ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                     num_batches_per_epoch=100, callbacks=[history]),
                )
            else:
                estimator = FF_gluonts_univariate_linear(
                    mean_layer,
                    distr_output=StudentTOutput(),
                    prediction_length=self.prediction_length,
                    context_length=self.context_length,
                    trainer=Trainer(hybridize=False,ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                     num_batches_per_epoch=100, callbacks=[history]),
                )
        elif self.model == 'transformer':
          estimator = Transformer_gluonts_gas_means(
              mean_layer,
              freq=self.freq,
              context_length=self.context_length,
              prediction_length=self.prediction_length,
              inner_ff_dim_scale= params['inner_ff_dim_scale'],
              model_dim= params['model_dim'],
              embedding_dimension= params['embedding_dimension'],
              num_heads= params['num_heads'],
              dropout_rate= params['dropout_rate'],
              trainer=Trainer(hybridize=False,ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                     num_batches_per_epoch=100, callbacks=[history]),
          )
        elif self.model == 'deepar' and not self.multivariate and isinstance(trained_mean_layer, GASNormalizer):
          estimator = DeepAR_gluonts_univariate_gas(
              mean_layer,
              freq=self.freq,
              distr_output=StudentTOutput(),
              context_length=self.context_length,
              prediction_length=self.prediction_length,
            #   num_cells= params['num_cells'],
            #   num_layers= params['num_layers'],
              trainer=Trainer(hybridize=False,ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                     num_batches_per_epoch=100, callbacks=[history]),
            #   trainer=Trainer(ctx=self.ctx,epochs=50, learning_rate=1e-4, num_batches_per_epoch=100),
          )

        ## TRAIN
        predictor = estimator.train(self.train, self.validation)
        ## EVALUATE
        if not save:
            test = self.validation
        else:
            test = self.test
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=test,  # test dataset
            predictor=predictor,  # predictor
            num_samples=100,  # number of sample paths we want for evaluation
        )

        forecasts = list(forecast_it)

        final_forecasts = []
        for f in forecasts:
            final_forecasts.append(f.median)

        mase_metrics = []
        for item_id, ts in enumerate(test):
          training_data = ts["target"].T[:-self.prediction_length]
          ground_truth = ts["target"].T[-self.prediction_length:]

          y_pred_naive = np.array(training_data)[:-int(self.seasonality)]
          mae_naive = mean_absolute_error(np.array(training_data)[int(self.seasonality):], y_pred_naive, multioutput="uniform_average")

          mae_score = mean_absolute_error(
              np.array(ground_truth),
              final_forecasts[item_id],
              sample_weight=None,
              multioutput="uniform_average",
          )

          epsilon = np.finfo(np.float64).eps
          if mae_naive == 0:
            continue
          mase_score = mae_score / np.maximum(mae_naive, epsilon)


          mase_metrics.append(mase_score)

        
        print("MINE")
        print(np.mean(mase_metrics))

        # print("GLUONTS")
        
        # tss = list(ts_it)
        # if self.multivariate:
        #     evaluator = MultivariateEvaluator(quantiles=[0.1, 0.5, 0.9])
        # else:
        #     evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

        # agg_metrics, item_metrics = evaluator(tss, forecasts)  # type: ignore # we are sure that tss is a list of DataFrame in multivariate case
        # # print(json.dumps(agg_metrics, indent=4))
        # # print(agg_metrics)
        # print(item_metrics.mean())

        if not save:
            return np.mean(mase_metrics)
      
        # make directory called saved_nonorm_{self.model}_{self.dataset_name}
        dir_name = f'saved_POINT_GAS_{self.model}_{self.dataset_name}_mean_str_{self.mean_str}_var_str_{self.var_str}'
        os.makedirs(dir_name, exist_ok=True)

        return np.mean(mase_metrics), predictor, dir_name, history



In [15]:
DATASET_NAME, model_choice, ctx, DATASET_FILE_FOLDER, n_trials, mean_str, var_str = 'nn5_weekly', 'feedforward', 'gpu', None, 1, 0.1, 0.1
multivariate = False
study = optuna.create_study(direction="minimize")
obj = Objective(
        model_choice,DATASET_NAME, ctx, DATASET_FILE_FOLDER, multivariate, mean_str, var_str
    )
study.optimize(
    obj,
    n_trials=n_trials,
)

[I 2024-02-09 13:25:15,001] A new study created in memory with name: no-name-eb3d6be8-96a8-4ded-bebe-6074dd71d523


Warming up train dataset...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 144 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 144 concurrent workers.


Warming up test dataset...


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.6s finished


...

Done.
Evaluating the estimator...
Done.


Running evaluation: 3it [00:11,  3.74s/it]
/tmp/ipykernel_143747/4189682787.py:108: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-5, 1e-1),


  item_id         forecast_start       MSE  abs_error  abs_target_sum  \
0    None  1998-03-23/1998-03-29  0.012700   0.636357       12.471882   
1    None  1998-03-23/1998-03-29  0.030339   0.854442        9.470610   
2    None  1998-03-23/1998-03-29  0.021980   0.988514       11.196161   

   abs_target_mean  seasonal_error      MASE      MAPE     sMAPE  \
0         1.558985        0.108730  0.731579  0.049151  0.051098   
1         1.183826        0.122170  0.874232  0.086990  0.098566   
2         1.399520        0.119135  1.037178  0.088496  0.091115   

   num_masked_target_values        ND       MSIS  QuantileLoss[0.1]  \
0                       0.0  0.051023  29.263178           0.339270   
1                       0.0  0.090220  34.969294           0.171291   
2                       0.0  0.088290  41.487134           0.640791   

   Coverage[0.1]  QuantileLoss[0.5]  Coverage[0.5]  QuantileLoss[0.9]  \
0          0.250           0.636357          0.250           0.933444   
1  

100%|██████████| 100/100 [00:03<00:00, 27.52it/s, epoch=1/13, avg_epoch_loss=0.1]
1it [00:00, 70.86it/s, epoch=1/13, validation_avg_epoch_loss=0.0667]
100%|██████████| 100/100 [00:03<00:00, 28.17it/s, epoch=2/13, avg_epoch_loss=0.0913]
1it [00:00, 70.69it/s, epoch=2/13, validation_avg_epoch_loss=0.0738]
100%|██████████| 100/100 [00:03<00:00, 27.69it/s, epoch=3/13, avg_epoch_loss=0.0868]
1it [00:00, 80.70it/s, epoch=3/13, validation_avg_epoch_loss=0.0839]
100%|██████████| 100/100 [00:03<00:00, 28.49it/s, epoch=4/13, avg_epoch_loss=0.0873]
1it [00:00, 82.38it/s, epoch=4/13, validation_avg_epoch_loss=0.0705]
100%|██████████| 100/100 [00:03<00:00, 27.82it/s, epoch=5/13, avg_epoch_loss=0.0892]
1it [00:00, 90.89it/s, epoch=5/13, validation_avg_epoch_loss=0.0775]
100%|██████████| 100/100 [00:03<00:00, 27.51it/s, epoch=6/13, avg_epoch_loss=0.0908]
1it [00:00, 78.87it/s, epoch=6/13, validation_avg_epoch_loss=0.0791]
100%|██████████| 100/100 [00:03<00:00, 27.02it/s, epoch=7/13, avg_epoch_loss=0.

MINE
0.3241345469404516
